# Improved Search for Large Single Document with Highlights

This notebook demonstrates a streamlined approach to document question-answering using Highlights:

1. **Efficiency**: Processes large documents without sending the entire content to a frontier LLM
2. **Precision**: Uses Highlights' contextual awareness to identify relevant passages
3. **Cost-effectiveness**: Reduces token usage by focusing only on relevant sections

The implementation follows these steps:
1. Convert a PDF document to text
2. Split the text into page-level chunks
3. Use Highlights API to retrieve contextually relevant chunks
4. Send only the most relevant chunks to OpenAI for response generation

## Setup

Install the required libraries.

In [ ]:
!pip install PyPDF2 openai python-dotenv

import os
import PyPDF2
import openai
from dotenv import load_dotenv
from typing import List, Dict
from base_client import HighlightsClient
from utils import PDFProcessor

## Loading Environment Variables

Create a .env file with your API keys:
```
HIGHLIGHTS_API_KEY=your-highlights-api-key
OPENAI_API_KEY=your-openai-api-key
```

In [ ]:
# Load environment variables
load_dotenv()

# Initialize clients
highlights_client = HighlightsClient(api_key=os.getenv('HIGHLIGHTS_API_KEY'))
openai.api_key = os.getenv('OPENAI_API_KEY')

## PDF Processing

Extracting text from PDF documents presents several challenges including preserving formatting and structure. The PDFProcessor class handles this complexity while maintaining appropriate chunk boundaries for optimal retrieval.

In [ ]:
# Initialize processor
processor = PDFProcessor(highlights_client)

# Path to your PDF file
pdf_path = 'data/border_act.pdf'

# Extract text chunks from PDF
text_chunks = processor.extract_text_from_pdf(pdf_path)

print(f"Extracted {len(text_chunks)} pages from PDF")
print("\nSample from first page:")
print(text_chunks[0][:200] + "...")

## Contextual Retrieval with Highlights

Unlike vector search that treats chunks in isolation, Highlights evaluates each segment within its surrounding context, significantly improving retrieval relevance for large, complex documents.

In [ ]:
query = "Am I an eligible individual for CONDITIONAL PERMANENT RESIDENT STATUS? I was paroled into the us in 2020."

# Search for relevant chunks
relevant_chunks = processor.search_relevant_chunks(
    query=query,
    text_chunks=text_chunks,
    top_n=5  # Limiting to top 5 chunks balances completeness with token efficiency
)

print(f"Found {len(relevant_chunks)} relevant chunks")


## Response Generation

By forwarding only the most relevant chunks to a frontier model, we achieve two key benefits:
1. Reduced token consumption and lower costs
2. Higher quality responses by eliminating distracting or irrelevant content

In [ ]:
# Generate response using OpenAI
response = processor.generate_response(
    query=query,
    context=relevant_chunks
)

print("\nGenerated Response:")
print(response)